In [1]:
import pyxdf
import numpy as np
from sys import argv
from pathlib import Path

In [2]:
xdf_data_path = Path("xdf_data")

In [3]:
xdf_filepaths = list(xdf_data_path.rglob("*.xdf"))

In [4]:
xdf_data = {str(x).split("/")[1] : pyxdf.load_xdf(str(x)) for x in xdf_filepaths}

Stream 3: Calculated effective sampling rate 30.1718 Hz is different from specified rate 90.0000 Hz.
Stream 4: Calculated effective sampling rate 30.1676 Hz is different from specified rate 90.0000 Hz.
Stream 5: Calculated effective sampling rate 30.0871 Hz is different from specified rate 90.0000 Hz.
Stream 5: Calculated effective sampling rate 30.1664 Hz is different from specified rate 90.0000 Hz.
Stream 4: Calculated effective sampling rate 30.1437 Hz is different from specified rate 90.0000 Hz.
Stream 4: Calculated effective sampling rate 30.1845 Hz is different from specified rate 90.0000 Hz.
Stream 4: Calculated effective sampling rate 30.1260 Hz is different from specified rate 90.0000 Hz.
Stream 2: Calculated effective sampling rate 30.1270 Hz is different from specified rate 90.0000 Hz.
Stream 5: Calculated effective sampling rate 30.1097 Hz is different from specified rate 90.0000 Hz.
Stream 3: Calculated effective sampling rate 30.1637 Hz is different from specified rate 90

In [31]:

def separate_trials_expmakers(exp_list):
    for i, string in enumerate(exp_list[1:]):
        if string == "task_block_start|BOX_BLOCK":
            return [exp_list[0:i], exp_list[i:]], False
        elif string == "task_block_start|JEBSEN_TAYLOR":
            return [exp_list[i:], exp_list[0:i]], True
    # If no empty string is found, return the whole list as the first trial, second is empty
    raise Exception(f"Separate exp: {exp_list}")

def separate_trials_latmarkers(lat_list):
    for i, string in enumerate(lat_list[1:]):
        if string == "repetition_start|2": # Do I need to do a similar logic like above?
            return lat_list[0:i], lat_list[i:]
    # If the marker is not found, return the whole list as first trial, second is empty
    raise Exception(f"Separate lat: {lat_list}")

In [32]:
def collect_participant_trials(pnum):
    participant_trials = {
        "move_cans": {},
        "move_blocks": {}
    }

    exp_trial_0, exp_trial_1 = None, None
    lat_trial_0, lat_trial_1 = None, None
    trials_swapped = False
    exp_markers_reached = False
    lat_markers_reached = False
    
    data, header = xdf_data[f"sub-{pnum:03}"]  # Assuming xdf_data is already loaded
    for stream in data:
        stream_series = stream["time_series"]
        stream_name = stream["info"]["name"][0]
        stream_series_list = list(stream_series)
        
        if stream_name == "LatencyMarkers":
            if lat_markers_reached:
                raise Exception("Duplication of latency markers!")
            lat_markers_reached = True
            
            stream_series_np = np.array(stream_series).flatten()
            lat_trial_0, lat_trial_1 = separate_trials_latmarkers(list(stream_series_np))
            if exp_markers_reached:
                if trials_swapped:
                    participant_trials["move_cans"]["exp_markers"] = exp_trial_1
                    participant_trials["move_blocks"]["exp_markers"] = exp_trial_0
                    participant_trials["move_cans"]["lat_markers"] = lat_trial_1
                    participant_trials["move_blocks"]["lat_markers"] = lat_trial_0
                else:
                    participant_trials["move_cans"]["exp_markers"] = exp_trial_0
                    participant_trials["move_blocks"]["exp_markers"] = exp_trial_1
                    participant_trials["move_cans"]["lat_markers"] = lat_trial_0
                    participant_trials["move_blocks"]["lat_markers"] = lat_trial_1
            
        elif stream_name == "ExpMarkers" and not exp_markers_reached:
            exp_markers_reached = True
            
            stream_series_np = np.array(stream_series).flatten()
            # Not swapped can trial and then block trial
            [exp_trial_0, exp_trial_1], trials_swapped = separate_trials_expmakers(list(stream_series_np))

            if lat_markers_reached:
                if trials_swapped:
                    participant_trials["move_cans"]["exp_markers"] = exp_trial_1
                    participant_trials["move_blocks"]["exp_markers"] = exp_trial_0
                    participant_trials["move_cans"]["lat_markers"] = lat_trial_1
                    participant_trials["move_blocks"]["lat_markers"] = lat_trial_0
                else:
                    participant_trials["move_cans"]["exp_markers"] = exp_trial_0
                    participant_trials["move_blocks"]["exp_markers"] = exp_trial_1
                    participant_trials["move_cans"]["lat_markers"] = lat_trial_0
                    participant_trials["move_blocks"]["lat_markers"] = lat_trial_1

    if not lat_markers_reached or not exp_markers_reached:
        raise Exception("Some of the required markers were not reached!")
    
    return participant_trials

In [33]:
participant_data = collect_participant_trials(6)

In [34]:
for key in participant_data.keys():
    print(f"{key} with values {participant_data[key].keys()}")

move_cans with values dict_keys(['exp_markers', 'lat_markers'])
move_blocks with values dict_keys(['exp_markers', 'lat_markers'])


In [35]:
participant_data["move_cans"]

{'exp_markers': [np.str_('task_block_start|BOX_BLOCK'),
  np.str_('practice_session_start'),
  np.str_('labrecorder_start'),
  np.str_('practice_start'),
  np.str_('countdown_start'),
  np.str_('countdown_3'),
  np.str_('countdown_2'),
  np.str_('countdown_1'),
  np.str_('practice_boxblock_start'),
  np.str_('practice_boxblock_end'),
  np.str_('practice_end'),
  np.str_('labrecorder_stop'),
  np.str_('practice_session_end'),
  np.str_('labrecorder_start'),
  np.str_('trial_start'),
  np.str_('countdown_start'),
  np.str_('countdown_3'),
  np.str_('countdown_2'),
  np.str_('countdown_1'),
  np.str_('boxblock_start'),
  np.str_('block_moved'),
  np.str_('boxblock_end'),
  np.str_('task_accept|1'),
  np.str_('trial_end'),
  np.str_('labrecorder_stop'),
  np.str_('labrecorder_start'),
  np.str_('trial_start'),
  np.str_('countdown_start'),
  np.str_('countdown_3'),
  np.str_('countdown_2'),
  np.str_('countdown_1'),
  np.str_('boxblock_start'),
  np.str_('block_moved'),
  np.str_('block_mo